In [85]:
%pip install pdf2image google-cloud-storage pypdf --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.document_loaders import GCSFileLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
import glob

# To store the documents across all data sources:
all_documents = []

# 1. Load the .pdf document:


In [2]:
loader = PyPDFLoader("data/principles_of_marketing_book.pdf")
pages = loader.load_and_split()

print(pages[0])

page_content='Principles of Mark eting' metadata={'source': 'data/principles_of_marketing_book.pdf', 'page': 0}


In [3]:
# Add extra metadata to each page:
for page in pages:
    page.metadata["description"] = "Principles of Marketing Book"

In [4]:
# Checking that the metadata has been added:
for page in pages[0:2]:
    print(page.metadata)

# Saving the marketing book pages:
all_documents.extend(pages)

{'source': 'data/principles_of_marketing_book.pdf', 'page': 0, 'description': 'Principles of Marketing Book'}
{'source': 'data/principles_of_marketing_book.pdf', 'page': 1, 'description': 'Principles of Marketing Book'}


# 2. Search the data folder for all of the .csv files:


In [5]:
csv_files = glob.glob("data/*.csv")

# Filter to only include the word Marketing in the file name:
csv_files = [f for f in csv_files if "Marketing" in f]

# For each .csv file:
for csv_file in csv_files:
    loader = CSVLoader(file_path=csv_file)
    data = loader.load()
    # Saving the data to the all_documents list:
    all_documents.extend(data)

# 3. Extracting 3 marketing reports from Google Drive.


In [6]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=200, chunk_overlap=0
)

blobs = [
    "NutriFusion Foods Marketing Plan 2022.docx",
    "NutriFusion Foods Marketing Plan 2023.docx"
]

docs = []
for blob in blobs:
    loader = GCSFileLoader(project_name="vexpower-2b2c5", bucket="oreilly-content", blob=blob)
    pages = loader.load()
    chunks = text_splitter.split_documents(pages)
    docs.extend(chunks)

# Saving the marketing book pages:
all_documents.extend(docs)

/Users/jamesaphoenix/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/jamesaphoenix/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
